In [36]:
!pip install selenium

In [ ]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver-manager

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (89.0.4389.90-0ubuntu0.18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import json
# Set driver for Chrome
options = webdriver.ChromeOptions()
options.add_argument('-headless') # since we run selenium on Google Colab so we don't want a chrome browser opens, so it will run in the background
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument("--incognito")

In [ ]:
def selenium_scrap(tiki_url):
  driver = webdriver.Chrome('chromedriver',options=options)        # Define the chrome drivers with setting options we define above  
  driver.implicitly_wait(60)                                       # We let selenium to wait for 30 seconds for all javascript script done before return the result of HTML
                                     
  driver.get(tiki_url)                                             # Open the browser again to get web page
  html_data = driver.page_source                                   # After driver.get() is done, you can get back HTML string by using .page_source
  driver.close()                                                   # Close the driver after retrieving the web page
                                        

  soup = BeautifulSoup(html_data, 'html.parser')                   # do your beautifulsoup business like the usual
  products_a = soup.find_all('a', {'class':'product-item'})
  jasonproduct = soup.find_all('script', {"type":"application/ld+json"})
  print('Total product in page: ',len(products_a)) #numbers of product.
  page_product = []

  for item in range(len(products_a)):
    product_info_dict = {}
    each_product_thumbnail_info = products_a[item].div.find_all('div')
      # product image url
    product_image_url = each_product_thumbnail_info[0].img['src']
    product_info_dict['Image url'] = product_image_url
      # product class info
    product_name = products_a[item].find('div',{'class':'name'}).text
    product_info_dict['Product name'] = product_name
    product_price_discount = products_a[item].find('div',{'class':'price-discount__price'}).text
    product_info_dict['Price after discount'] = product_price_discount
    
    try:
      assert products_a[item].find('div',{'class':'price-discount__discount'}).text.strip('-') != None
    except:
      product_info_dict['Discount'] = 'None'
    else:
      product_discount = products_a[item].find('div',{'class':'price-discount__discount'}).text.strip('-')
      product_info_dict['Discount'] = product_discount
      
    try:
      assert products_a[item].find('div',{'class':'review'}).text.strip('(').strip(')') != None
    except:
      product_info_dict['Review count'] = 0
    else:
      product_review = products_a[item].find('div',{'class':'review'}).text.strip('(').strip(')')
      product_info_dict['Review count'] = product_review

   
   
      #Product SKU
    a = jasonproduct[item+1].text
    bdict = json.loads(a)
    product_info_dict['sku'] = bdict['sku']
    product_info_dict['description'] = bdict['description']
    product_info_dict['URL'] = bdict['url']
    
      
      # Append to page_product
      
    page_product.append(product_info_dict)
    
  return page_product

In [ ]:
def scrapper_tiki(url):
    page_number = 1
    tiki_url = url


    first,second = tiki_url.split('?')

    category_scrap = []
    while True:
      page = f'?page={page_number}'
      tiki_url = first+page+second
      page_product = selenium_scrap(tiki_url)
      if len(page_product) == 0:
        break
      category_scrap.extend(page_product)
      page_number+=1
      time.sleep(5)


    data = pd.DataFrame(data = category_scrap, columns = category_scrap[0].keys())

    # it should store a "result.csv" in your current folder, you can download it to store it
    data.to_csv("./result.csv", index=False)

In [ ]:
scrapper_tiki('https://tiki.vn/laptop/c8095?&src=static_block')

NameError: ignored